In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## CONFIG

In [2]:
from transformers import AutoTokenizer
import random
import os 
import torch
import numpy as np

class CFG():
    model_name = "binhquoc/vie-deberta-small"
    lr = 3e-4
    weight_decay = 0.01
    dropout = 0.1
    lamda = 0.9
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    epoch = 3
    seed = 42
    train_batch_size = 32
    eval_batch_size = 64
    device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
    max_length = 512,

cfg = CFG()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=cfg.seed)

## Load and process data

In [3]:
import json

def read_json(path):
    with open(path, "r", encoding='utf-8') as f:
        json_data = json.load(f)
    return json_data

train_json = read_json('data/train.json')

In [4]:
intent_label = open('data/intent_label.txt', 'r').readlines()
intent_label = [i.replace('\n', '') for i in intent_label]

slot_label = open('data/slot_label.txt', 'r').readlines()
slot_label = [i.replace('\n', '') for i in slot_label]

cfg.intent2id = {intent:idx for idx, intent in enumerate(intent_label)}
cfg.slot2id = {slot:idx for idx, slot in enumerate(slot_label)}
cfg.id2intent = {idx:intent for idx, intent in enumerate(intent_label)}
cfg.id2slot = {idx:slot for idx, slot in enumerate(slot_label)}

In [5]:
index = 0
temp_data = train_json[index]
print('intent: ' + temp_data["intent"])
tokens = temp_data["text"].split()
slot_tags = temp_data["slot"].split()

line1 = ""
line2 = ""
for word, label in zip(tokens, slot_tags):
    max_length = max(len(word), len(label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += label + " " * (max_length - len(label) + 1)

print(line1)
print(line2)

intent: schedule
bạn cho cô Tháng hay lịch vào ngày   chín   âm     tháng   rồi     
O   O   O  O     O   O    O   B-date I-date I-date B-month I-month 


In [6]:
'B-round_trip'.startswith('B-')

True

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "binhquoc/vie-deberta-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
inputs = tokenizer(tokens, is_split_into_words=True)
print(inputs.tokens())
print(inputs.word_ids())

['[CLS]', '▁bạn', '▁cho', '▁cô', '▁Tháng', '▁hay', '▁lịch', '▁vào', '▁ngày', '▁chín', '▁âm', '▁tháng', '▁rồi', '[SEP]']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]


In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            label_name = cfg.id2slot[label]
            if label_name.startswith('B-'):
                label += 1
            new_labels.append(label)

    return new_labels

def tags2labels(tags, labels_mapping):
    return [labels_mapping[tag] for tag in tags]

In [10]:
slot_tags_labels = tags2labels(slot_tags , cfg.slot2id)
align_labels_with_tokens(slot_tags_labels, inputs.word_ids())

[-100, 0, 0, 0, 0, 0, 0, 0, 10, 11, 11, 14, 15, -100]

In [11]:
from torch.utils.data import Dataset

class WorldCupDataset(Dataset):
    def __init__(self, cfg, data_json, is_train=False):
        super().__init__()
        # if cfg.model_name in ['vinai/phobert-base', 'vinai/phobert-large']:
        #     segmentator = Segmentation()
        #     data_json = [merge_slot_after_segmentation(item, segmentator) for item in data_json]
            
        self.slot2id = cfg.slot2id
        self.intent2id = cfg.intent2id
        self.tokenizer = cfg.tokenizer
        self.cfg = cfg
        self.is_train = is_train
        self.tokenized_data = [self.process_data(data) for data in data_json]
        
    def  __len__(self):
        return len(self.tokenized_data)

    def process_data(self, data):
        inputs = self.tokenizer(data['text'])
        if self.is_train:
            if self.tokenizer.is_fast:
                intent_label = self.intent2id[data['intent']] if data['intent'] in self.intent2id else self.intent_dict['UNK']
                slot_tags = data["slot"].split()
                slot_labels = tags2labels(slot_tags , self.slot2id)
                new_slot_labels = align_labels_with_tokens(slot_labels, inputs.word_ids())
                inputs['intent_label'] = intent_label
                inputs['slot_labels'] = new_slot_labels
            else:
                raise('Non-fast tokenizer not processed yet')
        del inputs['token_type_ids']
        return inputs
    
    def align_labels_with_tokens(labels, word_ids):
        new_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id != current_word:
                # Start of a new word!
                current_word = word_id
                label = -100 if word_id is None else labels[word_id]
                new_labels.append(label)
            elif word_id is None:
                # Special token
                new_labels.append(-100)
            else:
                # Same word as previous token
                label = labels[word_id]
                # If the label is B-XXX we change it to I-XXX
                label_name = cfg.id2slot[label]
                if label_name.startswith('B-'):
                    label += 1
                new_labels.append(label)

        return new_labels

    def tags2labels(tags, labels_mapping):
        return [labels_mapping[tag] if tag in labels_mapping else labels_mapping['UNK'] for tag in tags]

    def __getitem__(self, idx):
        return self.tokenized_data[idx]
    
################################
wc_train_dataset = WorldCupDataset(cfg, train_json[:int(0.8*len(train_json))], True)
wc_eval_dataset = WorldCupDataset(cfg, train_json[int(0.8*len(train_json)):], True)
# wc_test_dataset = WorldCupDataset(cfg, train_json, False)

In [12]:
wc_train_dataset[0]

{'input_ids': [5, 126, 17, 211, 1750, 168, 438, 37, 59, 2296, 730, 136, 259, 4], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'intent_label': 2, 'slot_labels': [-100, 0, 0, 0, 0, 0, 0, 0, 10, 11, 11, 14, 15, -100]}

In [13]:
class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = {k:[sample[k] for sample in batch] for k in batch[0].keys()}

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
        output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        if "slot_labels" in output:
            output["slot_labels"] = [s + (batch_max - len(s)) * [-100] for s in output["slot_labels"]]

        # convert to tensors
        output = {k:torch.tensor(v, dtype=torch.long) for k,v in output.items()}

        return output

collate_fn = Collate(cfg.tokenizer)

In [14]:
out_collator = collate_fn(wc_train_dataset[0:2])
out_collator

{'input_ids': tensor([[    5,   126,    17,   211,  1750,   168,   438,    37,    59,  2296,
            730,   136,   259,     4,     0,     0,     0,     0,     0,     0,
              0,     0],
         [    5,  1194,    17,  1983,   347,  3489,   109,   140,   205,   263,
              9,   313,   197,   704, 13222,  3024,    59,  2669,  2338,    43,
            171,     4]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'intent_label': tensor([2, 5]),
 'slot_labels': tensor([[-100,    0,    0,    0,    0,    0,    0,    0,   10,   11,   11,   14,
            15, -100, -100, -100, -100, -100, -100, -100, -100, -100],
         [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    2,    3,   10,   11,   11,    0,    0, -100]])}

## Full Model

In [15]:
import torch 
import torch.nn as nn
import numpy as np

class Attention(nn.Module):
    """Applies attention mechanism on the `context` using the `query`.
    Args:
        dimensions (int): Dimensionality of the query and context.
        attention_type (str, optional): How to compute the attention score:
    """
    def __init__(self, hidden_size = 256):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.softmax = nn.Softmax(dim=1)
        self.tanh = nn.Tanh()

    def forward(self, query, context, attention_mask):
        if not (query.size()[-1] == self.hidden_size and context.size()[-1] == self.hidden_size):
            raise Exception("Dimensions must be equal")

        attention_scores = torch.bmm(query, context.transpose(1, 2).contiguous())
        if attention_mask is not None:
            attention_mask = torch.unsqueeze(attention_mask, 2)
            attention_scores.masked_fill_(attention_mask == 0, - np.inf)

        attention_weights = self.softmax(attention_scores)
        context_vector = torch.bmm(attention_weights.transpose(1, 2), query)
        combined = torch.add(context_vector, context)
        output = self.tanh(combined)

        return output, attention_weights

class IntentClassifier(nn.Module):
    def __init__(self, input_dim = 256, num_intent_labels = 25, dropout_rate=0.0):
        super(IntentClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, num_intent_labels)
        self.__init_fc_params()
    
    def __init_fc_params(self):
        nn.init.xavier_normal_(self.linear.weight)
        nn.init.constant_(self.linear.bias, 0)

    def forward(self, intent_input):
        intent_input = self.dropout(intent_input)

        return self.linear(intent_input)


class SlotClassifier(nn.Module):
    def __init__(
        self,
        input_dim = 768,
        num_slot_labels = 142,
        attention_embedding_size = 256,
        dropout_rate = 0.0,
    ):
        super(SlotClassifier, self).__init__()

        self.attention = Attention(attention_embedding_size)
        self.linear_slot = nn.Linear(input_dim, attention_embedding_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear_out = nn.Linear(attention_embedding_size, num_slot_labels)
        self.__init_fc_params()
    
    def __init_fc_params(self):
        nn.init.xavier_normal_(self.linear_slot.weight)
        nn.init.constant_(self.linear_slot.bias, 0)

        nn.init.xavier_normal_(self.linear_out.weight)
        nn.init.constant_(self.linear_out.bias, 0)

    def forward(self, sequence_output, intent_context, attention_mask=None):
        sequence_output = self.linear_slot(sequence_output)         # (b, seq, hidden_size) -> (b, seq, att_size)
        intent_context = self.linear_slot(intent_context)           # (b, hidden_size) -> (b, att_size)
        intent_context = torch.unsqueeze(intent_context, 1)         # (b, att_size) -> (b, 1, att_size)
        
        attention_output, weights = self.attention(sequence_output, intent_context, attention_mask)     #(b, 1, att_size)
        intent_input = attention_output.squeeze(1)                  # (b, 1, att_size) -> #(b, att_size)
        sequence_output = torch.mul(sequence_output, attention_output)      #(b, seq, att_size)
        sequence_output = self.dropout(sequence_output)

        return intent_input, self.linear_out(sequence_output)

# if __name__ == '__main__':
#     slot = SlotClassifier(num_slot_labels=len(cfg.slot2id))
#     intent = IntentClassifier(num_intent_labels =len(cfg.intent2id))

#     sequence_output = torch.rand((2,50,768))
#     pooled_output = torch.rand((2, 768))
#     tmp_attention_mask = None

#     intent_in, slot_out = slot(sequence_output=sequence_output, intent_context=pooled_output, attention_mask=tmp_attention_mask)
#     intent_out = intent(intent_input=intent_in)

#     print("Intent_in size: ", intent_in.size())
#     print("Intent_out size: ", intent_out.size())
#     print("Slot_out size: ", slot_out.size())

In [16]:
from transformers import AutoModel, AutoConfig
class WorldCup(nn.Module):
    def __init__(self, cfg):
        super(WorldCup, self).__init__()
        self.model = AutoModel.from_pretrained(cfg.model_name)
        self.config_model = AutoConfig.from_pretrained(cfg.model_name)
        self.num_slot_labels = len(cfg.slot2id)
        self.num_intent_labels = len(cfg.intent2id)
        self.slot = SlotClassifier(input_dim = self.config_model.hidden_size, num_slot_labels = self.num_slot_labels, dropout_rate = cfg.dropout)
        self.intent = IntentClassifier(num_intent_labels = self.num_intent_labels, dropout_rate=cfg.dropout)
        self.loss_function = nn.CrossEntropyLoss(ignore_index=- 100)

    def forward(self, input_ids, attention_mask, intent_label = None, slot_labels = None):
        last_hidden_state = self.model(input_ids, attention_mask).last_hidden_state
        pooled_cls = last_hidden_state[:,0,:]

        # JointIDSF ver 1
        intent_in, slot_out = self.slot(sequence_output=last_hidden_state, intent_context=pooled_cls, attention_mask=attention_mask)
        intent_out = self.intent(intent_input=intent_in)
  
        if intent_label != None and slot_labels != None:
            #Cal loss
            intent_loss = self.loss_function(intent_out, intent_label)
            slot_loss = self.loss_function(slot_out.view(-1, self.num_slot_labels), slot_labels.view(-1))
            return {
                'intent_loss': intent_loss,
                'slot_loss': slot_loss, 
                'intent_logits': intent_out, 
                'slot_logits': slot_out
            }
        else:
            # just return logits
            return {
                'intent_logits': intent_out, 
                'slot_logits': slot_out
            }

worldcup_model = WorldCup(cfg)

Some weights of the model checkpoint at binhquoc/vie-deberta-small were not used when initializing DebertaV2Model: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# worldcup_model(**out_collator)

## DataLoader

In [18]:
from torch.utils.data import DataLoader
train_loader = DataLoader(wc_train_dataset, batch_size = cfg.train_batch_size, collate_fn = collate_fn, shuffle=True, pin_memory=True)
eval_loader = DataLoader(wc_eval_dataset, batch_size = cfg.eval_batch_size, collate_fn = collate_fn, pin_memory=True, drop_last= True)

## Optimizer and Scheduler

In [19]:
from torch.optim import AdamW
from transformers import get_scheduler

optimizer = AdamW(worldcup_model.parameters(), lr=cfg.lr)
num_update_steps_per_epoch = len(train_loader)
num_training_steps = cfg.epoch * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(num_training_steps/10),
    num_training_steps=num_training_steps,
)

## Train, Evaluate, Inference function

In [20]:
def slot_postprocess(slot_predictions, slot_labels):
    predictions = slot_predictions.detach().cpu().clone().numpy()
    labels = slot_labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[cfg.id2slot[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [cfg.id2slot[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_predictions, true_labels

def intent_postprocess(intent_predictions, intent_labels):
    predictions = intent_predictions.detach().cpu().clone().numpy()
    labels = intent_labels.detach().cpu().clone().numpy()
    
    true_labels = [cfg.id2intent[l]  for l in labels]
    true_predictions = [cfg.id2intent[l]  for l in predictions]
    return true_predictions, true_labels

In [21]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))
worldcup_model.to(cfg.device)
for epoch in range(cfg.epoch):
    # Training
    worldcup_model.train()
    for batch in train_loader:
        batch = {k:v.to(cfg.device, dtype = torch.long) for k,v in batch.items()}
        outputs = worldcup_model(**batch)
        loss = outputs['slot_loss']*cfg.lamda + outputs['intent_loss']*(1-cfg.lamda)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.set_postfix(
            Epoch = epoch,
            slot_loss = outputs['slot_loss'].item(),
            intent_loss = outputs['intent_loss'].item(),
            combined_loss = loss.item(),
            lr = optimizer.param_groups[0]['lr']
        )
        progress_bar.update(1)
    # Evaluation
    worldcup_model.eval()
    total_slot_eval_loss = 0
    total_intent_eval_loss = 0
    slot_predictions_list = []
    slot_labels_list = []
    intent_predictions_list = []
    intent_labels_list = []

    for batch in eval_loader:
        with torch.no_grad():
            batch = {k:v.to(cfg.device, dtype = torch.long) for k,v in batch.items()}
            outputs = worldcup_model(**batch)
        
        total_slot_eval_loss += outputs['slot_loss'].item()
        total_intent_eval_loss += outputs['intent_loss'].item()

        intent_predictions = outputs['intent_logits'].argmax(dim=-1)
        slot_predictions = outputs['slot_logits'].argmax(dim=-1)
        
        # Convert index to label name
        slot_predictions, slot_labels = slot_postprocess(slot_predictions, batch['slot_labels'])
        intent_predictions, intent_labels = intent_postprocess(intent_predictions, batch['intent_label'])
        

        # Compute metric (accuracy)
        slot_predictions_list += slot_predictions
        slot_labels_list += slot_labels
        intent_predictions_list += intent_predictions
        intent_labels_list += intent_labels
    
    # flatten results
    slot_predictions_list = [slot for slots in slot_predictions_list for slot in slots]
    slot_labels_list = [slot for slots in slot_labels_list for slot in slots]

    slot_accuracy = sum([(1 if p==l else 0)for p,l in zip(slot_predictions_list, slot_labels_list)])/len(slot_predictions_list)
    intent_accuracy = sum([(1 if p==l else 0)for p,l in zip(intent_predictions_list, intent_labels_list)])/len(intent_predictions_list)
    print(
        f"epoch {epoch}:",
        {
            'slot_eval_loss': total_slot_eval_loss/len(eval_loader),
            'intent_eval_loss': total_intent_eval_loss/len(eval_loader),
            'slot_accuracy':slot_accuracy,
            'intent_accuracy':intent_accuracy
        },
    )


  0%|          | 0/2250 [00:00<?, ?it/s]

/home/viet/miniconda3/envs/worldcup/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(


epoch 0: {'slot_eval_loss': 0.003985847205816147, 'intent_eval_loss': 0.003385974205679871, 'slot_accuracy': 0.9992992828598019, 'intent_accuracy': 0.9996639784946236}
epoch 1: {'slot_eval_loss': 0.0014737916695711675, 'intent_eval_loss': 0.0003321582275587985, 'slot_accuracy': 0.9997262823671101, 'intent_accuracy': 1.0}
epoch 2: {'slot_eval_loss': 0.0005013562409188917, 'intent_eval_loss': 0.0003020500561462775, 'slot_accuracy': 0.9998576668308973, 'intent_accuracy': 1.0}


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint/final_checkpoint'

In [30]:
torch.save(worldcup_model.state_dict(), 'checkpoints/final_checkpoint.bin')
tokenizer.save_pretrained('checkpoints')

('checkpoints/tokenizer_config.json',
 'checkpoints/special_tokens_map.json',
 'checkpoints/tokenizer.json')

In [31]:
from torch.nn.functional import softmax

# text = "Anh với Ý có đá với nhau không"
def inference(text):
    inputs = tokenizer(text, return_tensors = 'pt',return_offsets_mapping= True)
    del inputs['token_type_ids']
    worldcup_model.to('cpu')
    with torch.no_grad():
        outputs = worldcup_model(inputs['input_ids'], inputs['attention_mask'])

    intent_sm = softmax(outputs['intent_logits'], dim=-1)
    intent_score, intent_id = torch.max(intent_sm, dim = -1)

    score = softmax(outputs['slot_logits'], dim=-1)
    slot_score, slot_id= torch.max(score, dim = -1)

    intent_score = intent_score.detach().numpy()[0]
    slot_score = slot_score.detach().numpy()[0]
    intent_id = intent_id.detach().numpy()[0]
    slot_id = slot_id.detach().numpy()[0]
    # print(slot_id)

    intent_label = cfg.id2intent[intent_id]
    slot_label = [cfg.id2slot[id] for id in slot_id]

    offset_mapping = inputs['offset_mapping'].detach().numpy()[0]
    slot_list = []
    real_tag = None
    for i, tag in enumerate(slot_label):
        if tag.startswith("B-") and real_tag is None:
            start = offset_mapping[i][0]
            start = start + 1 if start != 0 else start # + 1 to offset of token with a space before
            real_tag = tag[2:]
            score = slot_score[i]
        elif tag.startswith("I-") and tag[2:] == real_tag:
            continue
        elif real_tag is not None:
            end = offset_mapping[i-1][1]
            slot_list.append({
                'slot_type': real_tag,
                'score': score,
                'slot_text':  text[start:end],
                'start': start,
                'end': end
            })
            real_tag = None
            if tag.startswith("B-"):
                start = offset_mapping[i][0]
                start = start + 1 if start != 0 else start # + 1 to offset of token with a space before
                real_tag = tag[2:]
                score = slot_score[i]
        
    return {
        'intent':{
            'intent_type': intent_label,
            'score': intent_score,
        },
        'slot': slot_list,
        'text': text,
    }
# Bug từ đầu tiên ("thứ 3 brazil đá mấy giờ"), "chủ nhật brazil đá mấy giờ", "chủ nhật brazil diến ra mấy giờ"
inference("xếp hạng của bồ đào nha hiện đang như thế nào vậy maika")


tensor([[-1.3577, -3.0024, -1.6577, -0.9406, -3.8050, -0.9548, -0.5570,  7.1578]])
tensor([[[ 5.5049e+00,  2.8271e-01,  8.3822e+00,  3.8825e+00, -2.6597e+00,
          -9.9495e-01, -2.4784e-01,  1.1292e+00,  7.4237e-01, -1.8747e+00,
          -1.8218e+00,  1.3861e+00, -1.8063e+00,  3.1873e-01, -3.2052e+00,
           1.7141e+00, -4.8388e-01, -1.0889e+00, -1.2534e+00, -1.0277e-01,
          -7.3484e-01, -1.3350e+00,  8.6325e-01, -8.3508e-01, -3.0208e-02,
           1.1886e+00],
         [ 1.5911e+01, -1.5504e+00,  3.0659e+00,  1.7730e+00, -6.1234e-01,
          -4.6469e-01,  8.3353e-01,  1.1889e+00, -2.6595e+00, -8.2903e-01,
          -4.5051e-01, -4.8398e-01, -2.5500e+00, -2.6374e+00, -1.2673e+00,
           6.4588e-01, -9.1391e-01, -3.6846e+00,  1.8230e+00, -5.7142e-02,
          -1.8725e+00, -5.9644e-01,  2.0268e+00,  6.2443e-02,  5.4360e-01,
          -2.6621e-01],
         [ 1.7715e+01, -1.6150e+00,  1.7190e+00,  2.1890e+00, -1.5864e+00,
          -1.0598e+00,  4.6863e-01,  4.9405e

/home/viet/miniconda3/envs/worldcup/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(


{'intent': {'intent_type': 'result.point', 'score': 0.9985475},
 'slot': [{'slot_type': 'team',
   'score': 0.9310429,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.9999249,
   'slot_text': 'bồ đào nha hiện',
   'start': 13,
   'end': 28}],
 'text': 'xếp hạng của bồ đào nha hiện đang như thế nào vậy maika'}

In [25]:
testcase = [
    "maika cho mình hỏi pháp và bồ đào nha khi nào đá với nhau vậy",
    'soi kèo brazil và tây ban nha đi maika',
    'cho tớ biết tỉ số anh và thụy điển vào hôm qua',
    'bảng a có kết quả như thế nào rồi',
    'đội tuyển anh có khả năng nhất không',
    'dự đoán anh và bỉ ngày mai ai thắng',
    'khi nào bảng f đá vậy',
    'xem lịch worldcup ngày mai',
    'kết quả trận đấu bỉ và pháp vào bao nhiêu quả'
]
for i in testcase:
    display(inference(i))

{'intent': {'intent_type': 'schedule.nearest', 'score': 0.9995115},
 'slot': [{'slot_type': 'team',
   'score': 0.8021553,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.9999466,
   'slot_text': 'pháp',
   'start': 19,
   'end': 23},
  {'slot_type': 'team',
   'score': 0.99998903,
   'slot_text': 'bồ đào nha',
   'start': 27,
   'end': 37}],
 'text': 'maika cho mình hỏi pháp và bồ đào nha khi nào đá với nhau vậy'}

{'intent': {'intent_type': 'infor.score', 'score': 0.99837565},
 'slot': [{'slot_type': 'team',
   'score': 0.999966,
   'slot_text': 'brazil',
   'start': 8,
   'end': 14},
  {'slot_type': 'team',
   'score': 0.9999974,
   'slot_text': 'tây ban nha',
   'start': 18,
   'end': 29}],
 'text': 'soi kèo brazil và tây ban nha đi maika'}

{'intent': {'intent_type': 'infor.score', 'score': 0.9998349},
 'slot': [{'slot_type': 'team',
   'score': 0.5846616,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.9995927,
   'slot_text': 'anh',
   'start': 18,
   'end': 21},
  {'slot_type': 'team',
   'score': 0.9999864,
   'slot_text': 'thụy điển',
   'start': 25,
   'end': 34},
  {'slot_type': 'day',
   'score': 0.99999714,
   'slot_text': 'hôm qua',
   'start': 39,
   'end': 46}],
 'text': 'cho tớ biết tỉ số anh và thụy điển vào hôm qua'}

{'intent': {'intent_type': 'infor.score', 'score': 0.9649062},
 'slot': [{'slot_type': 'team',
   'score': 0.6023939,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'branch',
   'score': 0.999985,
   'slot_text': 'bảng a',
   'start': 0,
   'end': 6}],
 'text': 'bảng a có kết quả như thế nào rồi'}

{'intent': {'intent_type': 'result.predict.winner', 'score': 0.88982755},
 'slot': [{'slot_type': 'team',
   'score': 0.80810153,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.999948,
   'slot_text': 'anh',
   'start': 10,
   'end': 13},
  {'slot_type': 'rank',
   'score': 0.9251872,
   'slot_text': 'nhất',
   'start': 26,
   'end': 30}],
 'text': 'đội tuyển anh có khả năng nhất không'}

{'intent': {'intent_type': 'infor.score', 'score': 0.99967086},
 'slot': [{'slot_type': 'team',
   'score': 0.75083566,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.9999443,
   'slot_text': 'anh',
   'start': 8,
   'end': 11},
  {'slot_type': 'team',
   'score': 0.9999559,
   'slot_text': 'bỉ',
   'start': 15,
   'end': 17},
  {'slot_type': 'day',
   'score': 0.99999857,
   'slot_text': 'ngày mai',
   'start': 18,
   'end': 26}],
 'text': 'dự đoán anh và bỉ ngày mai ai thắng'}

{'intent': {'intent_type': 'schedule', 'score': 0.99887854},
 'slot': [{'slot_type': 'branch',
   'score': 0.9999962,
   'slot_text': 'bảng f',
   'start': 8,
   'end': 14}],
 'text': 'khi nào bảng f đá vậy'}

{'intent': {'intent_type': 'schedule', 'score': 0.9998155},
 'slot': [{'slot_type': 'day',
   'score': 0.9999931,
   'slot_text': 'ngày mai',
   'start': 18,
   'end': 26}],
 'text': 'xem lịch worldcup ngày mai'}

{'intent': {'intent_type': 'infor.score', 'score': 0.9997377},
 'slot': [{'slot_type': 'team',
   'score': 0.82631934,
   'slot_text': '',
   'start': 0,
   'end': 0},
  {'slot_type': 'team',
   'score': 0.99997425,
   'slot_text': 'bỉ',
   'start': 17,
   'end': 19},
  {'slot_type': 'team',
   'score': 0.99997866,
   'slot_text': 'pháp',
   'start': 23,
   'end': 27}],
 'text': 'kết quả trận đấu bỉ và pháp vào bao nhiêu quả'}

In [32]:

# def Convert_ONNX(model, data_input, logs_dir = "./logs"):
# Export the model
inputs = tokenizer("This is a sample", return_tensors = 'pt',return_offsets_mapping= True)
del inputs['token_type_ids']
# os.makedirs(logs_dir, exist_ok=True)
torch.onnx.export(
    worldcup_model,               # model being run
    (inputs['input_ids'], inputs['attention_mask']), # model input (or a tuple for multiple inputs)
    f="worldcup-model.onnx",   # where to save the model (can be a file or file-like object)
    opset_version=13,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = ['input_ids', 'attention_mask'],   # the model's input names
    output_names = ['intent_logits', 'slot_logits'], # the model's output names
    dynamic_axes={'input_ids' : {0 : 'batch_size', 1: 'sequence_len'},    # variable length axes
                    'attention_mask' : {0 : 'batch_size', 1: 'sequence_len'},
                    'intent_logits' : {0 : 'batch_size', 1: 'intent_labels'},
                    'slot_logits' : {0 : 'batch_size', 1: 'sequence_len', 2: 'slot_labels'}}
)


/home/viet/miniconda3/envs/worldcup/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:744: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  scale = torch.sqrt(torch.tensor(query_layer.size(-1), dtype=torch.float) * scale_factor)
/home/viet/miniconda3/envs/worldcup/lib/python3.9/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:744: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale = torch.sqrt(torch.tensor(query_layer.size(-1), dtype=torch.float) * scale_factor)
/home/viet/miniconda3/envs/worldcup/lib/python3.9/site-packages/transform